In [19]:
from langchain.llms.bedrock import Bedrock
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
import boto3
from langchain.chains import LLMChain
import botocore
config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 0}
)


bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)

file_paths = [r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\One Pharma\test-1.pdf",
              r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\One Pharma\test-2.pdf",
              r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\One Pharma\test-3.pdf",
              r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\One Pharma\test-4.pdf",
              r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\One Pharma\test-5.pdf",
              r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\One Pharma\test-6.pdf"]

docs = []
for path in file_paths:
    loader = PyPDFLoader(path)
    docs.extend(loader.load())


template = """
    Generate answers strictly from the given documents only, i need all the related and relevant answers from the documents
        
        Instructions:
            - if the related or relavent answers available in multiple documents means print all the exact relavent results from all the documents
            - Do not generate answers from your knowledge
            - if there are multiple documents available means search for all the documents about asked question and display all the relevent and related results
            - Must print the results even the single information is present on any of the given document based on the asked question
            - Must print all the relevent results about the asked question{question}
            - strictly avoid or remove'\n' while printing the results
            - Must strictly print the document name(name of the file along with the results)
                for axample: if the query or question is about 'what is AI' or 'Tell me about AI' (if the answer is found on 2 documents means you should print like below)
                            document 1(filename:)
                                    Artificial intelligence (AI) is a branch of computer science that deals with the creation of intelligent agents, which are systems that can reason, learn, and act autonomously.
                            document 2(filename:)
                                    Artificial Intelligence (AI) refers to the simulation of human intelligence processes by machines, especially computer systems

                        - so these are the examples for how you should answers if multiple answers are present on the multiple documents
                        - if the answer is present on all the documents means you should print all the document's results along with the filename(metadata)
                        - file name is present in the end of each documemt's metadata
                            example: if the (metadata)source is 'C:\\Users\\Lenovo\\Documents\\Project-vs code\\Amazon Transcribe\\One Pharma\\ARTICLE.pdf', here the file name is 'ARTICLE.pdf'
                            so take the file name accurately

    For example:
        if question is about 'what is Galaxy?' means
                You should search for 'Galaxy' in all the provided documents and return the results from all the documents (if availble)


{datas}
{question}
"""

qa_prompt = PromptTemplate(template=template, input_variables=["datas","question"])
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 40000})
llm_chain = LLMChain(prompt=qa_prompt, llm=llm, verbose= False)

In [15]:
print(docs)

[Document(page_content="A contract is essentially a legally binding agreement between two or more parties. It \nlays out specific rights and obligations that each party involved has to follow. Think of \nit as a formal way to ensure everyone involved in the agreement is on the same \npage and understands what's expected of them.  \nHere's a breakdown of some key points about contracts:  \n\uf0b7 Legally enforceable:  This is wh at separates a contract from a regular agreement. If \nsomeone breaches the contract (fails to uphold their end of the deal), the injured \nparty can take legal action to seek compensation or enforce the terms of the \nagreement.  \n\uf0b7 Exchange of value:  There needs t o be a reason for the contract to exist. This \nusually involves an exchange of something valuable, such as money, goods, or \nservices. For instance, you might enter a contract with a plumber to fix your sink; \nyou'd be paying them money for their service.  \n\uf0b7 Formal vs. Informal:  Con

In [21]:
result = llm_chain.run(datas= docs,question = "tell me about contract")
print(result)

 Here are the relevant details about contracts from the given documents:

document 1(filename:test-1.pdf)
A contract is essentially a legally binding agreement between two or more parties. It 
lays out specific rights and obligations that each party involved has to follow. Think of 
it as a formal way to ensure everyone involved in the agreement is on the same 
page and understands what's expected of them.  

Key points about contracts:

● Legally enforceable: If someone breaches the contract (fails to uphold their end of the deal), the injured 
party can take legal action to seek compensation or enforce the terms of the 
agreement.  

● Exchange of value: There needs to be a reason for the contract to exist, usually involving an exchange of something valuable like money, goods 
or services. 

● Formal vs. Informal: Contracts can be written or verbal. However, written contracts are generally considered stronger evidence in case of a disagreement.

● Widespread use: Contracts are used i